# Crash course on `numba`


`numpy` is the default option when trying to speed up a sicentific program. However, sometimes it may not be fast enough and a more low-level option may be needed.

There are several options (`ctypes`, `pybind11`,`f2py`). Most of these require the knowledge of a second programming language (C, C++, Fortran or Julia). 

`numba` is different. It is a set of tools that - combined with numpy - can significantly speedup a program written in `python`. It relies on writing the code slightly differently and providing extra information on its structure.

`numba` is installed via 

`conda install numba`

or 

`pip install numba`

## Minimal examples

For our purposes, `numba` is exclusively a tool to **speed up** code, so we will focus on simple examples where, with limited effort, a spedup can be achieved. We will try to minimise tha mount of jargon needed and the number of options.

We will focus on the manipulation of numpy arrays, as they are the most common data source in scientific computing.


What `numba` does is to transform functions written in python into faster functions, pre-compiled in machine code. This is useful for

- functions that are used often throughout a code
- functions that rely on multiple loops 



Let' start with a simle function. It performs a periodic boundary check on a vector within a [0,L] box.


In [29]:

def pbc(r,L):
    for i in range(r.shape[0]):
        for k in range(r.shape[1]):
            if r[i,k] <0:
                r[i,k]+= L
            elif r[i,k]>L:
                r[i,k]-=L

    

In [80]:
import numpy as np
from time import time
L = 10.0
eps = 0.5
N = 10000
np.random.seed(1)
r = np.random.uniform(0,L+eps,(N,2))

start = time()
pbc(r,L)
end = time()
print("Elapsed",end-start)


Elapsed 0.023220300674438477


In [110]:
import numba as nb
#no-python jit with signature, specifying the types as defined within Numba:
# result_type(argument1_type, argument2_type)
@nb.njit( nb.void(nb.double[:,:], nb.double),nogil=True) 
def nb_pbc(r,L):
    for i in range(r.shape[0]):
        for k in range(r.shape[1]):
            if r[i,k] <0:
                r[i,k]+= L
            elif r[i,k]>L:
                r[i,k]-=L

In [114]:
np.random.seed(1)
r = np.random.uniform(0,L+eps,(N,2))
start = time()
nb_pbc(r,L)
end = time()
print("Elapsed",end-start)

Elapsed 0.0005142688751220703


Let's compare with `numpy`

In [104]:
np.random.seed(1)
r = np.random.uniform(0,L+eps,(N,2))

start = time()
r[r<0]+=L
r[r>L]-=L
end = time()
print("Elapsed",end-start)

Elapsed 0.0021309852600097656


**Exercise**

Use numba to speed up the function you wrote to deal with interparticle collisions.